# Generative Adversarial Networks for text generation

The idea is build a GAN with pytorch.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim  
from torchvision import transforms
from torch.utils.data import DataLoader
from torchmetrics import Recall, Precision
import matplotlib.pyplot as plt

Building first a RNN.

In [7]:
data = "Hello how are you?"

chars = list(set(data))
char_to_ix =  {char:i for i, char in enumerate(chars)}
ix_to_char =  {i:char for i, char in enumerate(chars)}

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        out, _ = self.rnn(x,h0)
        out = self.fc(out[:, -1, :])
        return out
model = RNNModel(1, 16, 1)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:
inputs = [char_to_ix[ch] for ch in data[:-1]] 
targets = [char_to_ix[ch] for ch in data[1:]] 

inputs = torch.tensor(inputs, dtype = torch.long).view(-1,1)

inputs = nn.functional.one_hot(inputs, num_classes=len(chars)).float()

targets = torch.tensor(targets, dtype=torch.long)

In [10]:
for epoch in range(10):
    model.train()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1)%10 == 0:
        print(f'Epoch {epoch +1}/100, Loss: {loss.item()} ')

RuntimeError: input.size(-1) must be equal to input_size. Expected 1, got 12

Now i continue with the GAN

In [11]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(seq_lenght, seq_length),
            nn.Sigmoid()
        )
    def ferward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(seq_lenght, seq_length),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
generator = Generator()
discriminator = Discriminator()

criterion = BCELoss()

optimizer_gen = torch.optim.Adam(generator.parameters(), lr=0.001)
optimizer_disc = torch.optim.Adam(discriminator.parameters(), lr=0.001)

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    for real_data in data:
        real_data = real_data.unsqueeze(0)
        noise = torch.rand((1, seq_length))
        disc_real = discriminator(real_data)
        fake_data = generator(noise)
        disc_fake = discriminator(fake_data.detach())
        loss_disc = criterion(disc_real, torch.ones_likes(disc_real)) +  criterion(disc_fake, torch.ones_likes(disc_fake))
        optimizer_disc.zero_grad()
        optimizer_disc.step()
        disc_fake = discriminator(fake_data)
        loss_gen = criterion(disc_fake, torch.ones_like(dics_fake))
        optimizer_gen.zero_grad()
        loss_gen.backward()
        optimizer_gen.step()
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch +1}/{num_epochs}:\t Generator loss: {loss_gen.item()}\t Discrimiator loss: {loss.disc.item()}")

print("\nReal data: ")
print(data[:5])

print("\nGenerated data: ")

for _ in range(5):
    noise = torch.rand((1, seq_length))
    generated_data = generator(noise)
    print(torch.round(generated_data.detach()))

